In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

data = pd.read_csv("../input/data.csv")

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
def test_it_RF(data_test):
    clf = RandomForestClassifier(n_jobs=-1, n_estimators=10)
    return cross_val_score(clf, data_test.drop('shot_made_flag', 1), data_test.shot_made_flag,
                           scoring='accuracy', cv=10)

In [ ]:
# define the sort & enumeration function
def sort_encode(df, field):
    ct = pd.crosstab(df.shot_made_flag, df[field]).apply(lambda x:x/x.sum(), axis=0)
    temp = list(zip(ct.values[1, :], ct.columns))
    temp.sort()
    new_map = {}
    for index, (acc, old_number) in enumerate(temp):
        new_map[old_number] = index
    new_field = field + '_new'
    df[new_field] = df[field].map(new_map)
    return new_field

In [ ]:
data.loc[:5, ['loc_x', 'loc_y', 'lon', 'lat']]

In [ ]:
data[['loc_x', 'loc_y', 'lon', 'lat']].describe()

In [ ]:
data['home_or_away'] = 99
data.loc[data.matchup.str.find('@')==-1, 'home_or_away'] = 1
data.loc[data.matchup.str.find('vs')==-1, 'home_or_away'] = 0

In [ ]:
data.loc[:5, ['opponent', 'matchup', 'home_or_away']]

In [ ]:
data.iloc[0:5,0:6]

In [ ]:
data.iloc[0:5,6:13]

In [ ]:
data.iloc[0:5,13:18]

In [ ]:
data.iloc[0:5,18:25]

In [ ]:
loc_x_range = np.arange(-260,260,10)
binned = pd.cut(data.loc_x,loc_x_range) # 将loc_x区间化
binned_df = pd.DataFrame(binned)
binned_df.columns = ['loc_x_bin']
data = pd.concat([data,binned_df],axis=1)

loc_y_range = np.arange(-50,850,50)
binned_y = pd.cut(data.loc_y,loc_y_range) # 区间化
binned_y_df = pd.DataFrame(binned_y)
binned_y_df.columns = ['loc_y_bin']
data = pd.concat([data,binned_y_df],axis=1)

In [ ]:
data['remain_time'] = data['minutes_remaining']*60 + data['seconds_remaining']
data['time'] = 99
data.loc[data.remain_time>5, 'time'] = 0
data.loc[data.remain_time<=5, 'time'] = 1

In [ ]:
data['season_start_year'] = data.season.str.split('-').str[0]
data['season_start_year'] = data['season_start_year'].astype(int)
data['season_new'] = 99
data.loc[data.season_start_year<2013, 'season_new'] = 0
data.loc[data.season_start_year>=2013, 'season_new'] = 1

In [ ]:
data['home_or_away'] = 99
data.loc[data.matchup.str.find('@')==-1, 'home_or_away'] = 0
data.loc[data.matchup.str.find('vs')==-1, 'home_or_away'] = 1

In [ ]:
feature_encode = ['action_type', 'combined_shot_type', \
                  'shot_type','shot_zone_area', 'shot_zone_basic', 'shot_zone_range',\
                  'opponent', \
                  'loc_y_bin', 'loc_x_bin']
feature = []
for i in feature_encode:
    temp = sort_encode(data, i)
    feature.append(temp)

In [ ]:
feature1 = ['shot_distance', 'time', 'season_new', 'period', 'home_or_away', 'shot_id', 'shot_made_flag']
for i in feature1:
    feature.append(i)

In [ ]:
df = data[feature].copy()
mode = df.action_type_new.mode()[0]
df.action_type_new.fillna(mode, inplace=True)

In [ ]:
data_cv = df.loc[~df.shot_made_flag.isnull()].drop('shot_id',axis=1)
data_flag = df.loc[~df.shot_made_flag.isnull(), 'shot_made_flag']

In [ ]:
f = ['action_type_new',
 'combined_shot_type_new',
 'shot_type_new',
 'shot_zone_area_new',
 'shot_zone_basic_new',
 'shot_zone_range_new',
 'opponent_new',
 'loc_y_bin_new',
 'loc_x_bin_new',
 'shot_distance',
 'time',
 'season_new',
 'period',
 'home_or_away']

In [ ]:
f_acc = []
for i in f:
    f_acc.append(test_it_RF(data_cv[[i, 'shot_made_flag']]).mean())
df_acc = pd.DataFrame([f, f_acc])
df_acc = df_acc.T
df_acc.columns = ['feature', 'accuracy']
df_acc.sort_values('accuracy', ascending=False)

In [ ]:
import seaborn as sns
estimators, scores = list(range(1, 100, 5)), []
for i in estimators:
    clf = RandomForestClassifier(n_jobs=-1, n_estimators=i, random_state=2016)
    x = cross_val_score(clf, data_cv.drop('shot_made_flag', 1), data_cv.shot_made_flag, \
                              scoring='log_loss', cv=10)
    scores.append(x)
x = [i for i in estimators for j in range(10)]
sns.boxplot(x, np.array(scores).flatten())

In [ ]:
depth, scores = list(range(1, 20, 1)), []
for i in depth:
    clf = RandomForestClassifier(n_jobs=-1, n_estimators=80, max_depth=i, random_state=2016)
    x = cross_val_score(clf, data_cv.drop('shot_made_flag', 1), data_cv.shot_made_flag,
                              scoring='log_loss', cv=10)
    scores.append(x)
x = [i for i in depth for j in range(10)]
sns.boxplot(x, np.array(scores).flatten())

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, n_estimators=80, max_depth=7, random_state=2016)
rf.fit(data_cv.drop('shot_made_flag',axis=1), data_cv.shot_made_flag)

In [ ]:
importance = pd.DataFrame(rf.feature_importances_, \
                          index=list(data_cv.drop('shot_made_flag',axis=1).columns))
importance.columns = ['importances']
importance.sort_values('importances', ascending=False)

In [ ]:
from sklearn.feature_selection import RFECV
estimator = RandomForestClassifier(n_jobs=-1, n_estimators=80, max_depth=7, random_state=2016)
selector = RFECV(estimator, step=1, cv=10, scoring='log_loss')
selector.fit(data_cv.drop('shot_made_flag',axis=1), data_cv.shot_made_flag)

In [ ]:
selector_rank = pd.DataFrame(selector.ranking_, \
                          index=list(data_cv.drop('shot_made_flag',axis=1).columns))
selector_rank.columns = ['rank']
selector_rank.sort_values('rank', ascending=True)

In [ ]:
feature1 = ['action_type_new','shot_distance', 'shot_zone_basic_new', 'loc_x_bin']
df_train = df.loc[~df.shot_made_flag.isnull(), feature1]
df_test = df.loc[df.shot_made_flag.isnull(), feature1]
train_flag = df.loc[~df.shot_made_flag.isnull(), 'shot_made_flag']
shot_id = df.loc[df.shot_made_flag.isnull(), 'shot_id']

In [ ]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=1)
train = imp.fit_transform(df_train)
test = imp.transform(df_test)

In [ ]:
estimators, scores = list(range(1, 100, 5)), []
for i in estimators:
    clf = RandomForestClassifier(n_jobs=-1, n_estimators=i, random_state=2016)
    x = cross_val_score(clf, train, train_flag, scoring='log_loss', cv=10)
    scores.append(x)
x = [i for i in estimators for j in range(10)]
sns.boxplot(x, np.array(scores).flatten())

In [ ]:
depth, scores = list(range(1, 20, 1)), []
for i in depth:
    clf = RandomForestClassifier(n_jobs=-1, n_estimators=80, max_depth=i, random_state=2016)
    x = cross_val_score(clf, train, train_flag, scoring='log_loss', cv=10)
    scores.append(x)
x = [i for i in depth for j in range(10)]
sns.boxplot(x, np.array(scores).flatten())

In [ ]:
clf = RandomForestClassifier(n_jobs=-1, n_estimators=80, max_depth=7, random_state=2016)
scores1 = cross_val_score(clf, train, train_flag, scoring='accuracy', cv=10)
scores2 = cross_val_score(clf, train, train_flag, scoring='log_loss', cv=10)
print('accrucy: %s'%scores1.mean(), 'log_loss: %s'%scores2.mean())

In [ ]:
rf1 = RandomForestClassifier(n_jobs=-1, n_estimators=80, max_depth=7, random_state=2016)
rf1.fit(train, train_flag)

importance1 = pd.DataFrame(rf1.feature_importances_, index=feature1)
importance1.columns = ['importances']
importance1.sort_values('importances', ascending=False)

In [ ]:
clf = RandomForestClassifier(n_jobs=-1, n_estimators=80, max_depth=7, random_state=2016)

# Train and predict
clf.fit(train, train_flag)
predictions = clf.predict_proba(test)

# convert to CSV
submission = pd.DataFrame({'shot_id': shot_id, 'shot_made_flag': predictions[:, 1]})
submission[['shot_id', 'shot_made_flag']].to_csv('submission_RF.csv', index=False)